In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix

In [2]:
###### would have to install pymesh and might take a little too much #########

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')



Connecting celiib@10.28.0.34:3306


In [8]:
segment_id = 648518346341393609
dict_key = dict(segmentation=2,segment_id=segment_id)
mesh = (ta3p100.CleansedMesh & dict_key).fetch()
mesh

array([(2, 648518346341393609, Decimal('0.35'), 466615, 931294, array([[203098.765625  , 201025.015625  ,  11656.66015625],
       [203106.015625  , 201024.625     ,  11581.84375   ],
       [203141.9375    , 200947.84375   ,  11581.54101562],
       ...,
       [462651.5625    , 246667.09375   ,  63784.89453125],
       [462678.15625   , 246518.0625    ,  63816.5234375 ],
       [462718.6875    , 246634.125     ,  63846.13671875]]), array([[197667, 197827, 197212],
       [199865, 199228, 200170],
       [198980, 199087, 198398],
       ...,
       [407010, 407536, 406918],
       [408254, 408111, 408374],
       [408111, 408269, 408374]], dtype=uint32))],
      dtype=[('segmentation', '<i8'), ('segment_id', '<i8'), ('decimation_ratio', 'O'), ('n_vertices', '<i8'), ('n_triangles', '<i8'), ('vertices', 'O'), ('triangles', 'O')])

In [9]:
original_triangles = mesh["triangles"][0]
original_triangles[[1,2,3,4,6]]

array([[199865, 199228, 200170],
       [198980, 199087, 198398],
       [198980, 198398, 198770],
       [198980, 198770, 198708],
       [197831, 197022, 197606]], dtype=uint32)

In [10]:
#get the labels
mesh_labels = (ta3p100.CoarseLabelFinal & dict_key).fetch()

mesh_labels

array([(2, 648518346341393609, Decimal('0.35'), 'computer_Auto', 'bcelii', datetime.datetime(2019, 2, 1, 12, 39, 11), array([10, 10, 10, ..., 10, 10, 10], dtype=uint8), array([10, 10, 10, ...,  3,  3,  3], dtype=uint8))],
      dtype=[('segmentation', '<i8'), ('segment_id', '<i8'), ('decimation_ratio', 'O'), ('author_original', 'O'), ('author_proofreader', 'O'), ('date_time', 'O'), ('vertices', 'O'), ('triangles', 'O')])

In [11]:
dict_key = dict(segmentation=2,segment_id=segment_id)
ta3p100.CoarseLabelFinal() & dict_key

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as ta3p100.ProofreadLabel
2,648518346341393609,0.35,computer_Auto,bcelii,2019-02-01 12:39:11,=BLOB=,=BLOB=


In [12]:
#look for errors
not_errors = [k for k in mesh_labels["triangles"][0] if k != 10]
print(len(not_errors))

651958


In [ ]:
len(original_triangles[not_errors])

In [ ]:
#see if has any error labels
traingles

In [ ]:
ta3p100.CoarseLabelFinal()

In [ ]:




def generate_neighborhood(self, triangles, num_vertices):
    neighborhood = dict()
    for i in range(num_vertices):
        neighborhood[i] = set()
    for node1, node2, node3 in triangles:
        neighborhood[node1].update([node2, node3])
        neighborhood[node2].update([node1, node3])
        neighborhood[node3].update([node1, node2])
    return neighborhood

def set_search_first(self, starting_node, neighborhood):
    """
    Modified Depth-First-Search utilizing sets to reduce duplicate checks:

    Neighborhood must be a dict with the keys being the vertex indices!
    """    
    visited_nodes = set()
    temp_stack = set()
    temp_stack.add(starting_node)
    while len(temp_stack) > 0:
        starting_node = temp_stack.pop()
        if starting_node not in visited_nodes:
            visited_nodes.add(starting_node)
            temp_stack.update(neighborhood[starting_node])
    return list(visited_nodes)

def get_connected_portions(self, neighborhood):
    neighborhood_copy = neighborhood.copy()
    portions = []
    while len(neighborhood_copy) > 0:
        starting_node = next(iter(neighborhood_copy))
        portion = self.set_search_first(starting_node, neighborhood_copy)
        for node in portion:
            neighborhood_copy.pop(node)
        portions.append(portion)
    return portions

def get_largest_portion_index(self, portions):
    portion_lengths = [len(portion) for portion in portions]
    return portion_lengths.index(max(portion_lengths))

def get_largest_portion(self, portions):
    return portions[self.get_largest_portion_index(portions)]

def remove_floating_artifacts(self, mesh):    
    mesh_copy = mesh.copy()

    """
    # Generating the neighborhoods gets quite expensive for full resolution meshes, but the searches are extremely quick.
    neighborhood = self.generate_neighborhood(mesh_copy['triangles'], len(mesh_copy['vertices']))
    portions = self.get_connected_portions(neighborhood)

    main_mesh_body_index = self.get_largest_portion_index(portions)
    triangle_removal_nodes = portions[main_mesh_body_index:] + portions[:main_mesh_body_index + 1]

    new_triangles = []
    main_body_portion = set(self.get_largest_portion(portions))
    for i, triangle in enumerate(mesh_copy['triangles']):
        node1 = triangle[0]
        if node1 in main_body_portion:
            new_triangles.append(triangle)
    mesh_copy['triangles'] = np.array(new_triangles)"""
    
    #get the triangle indices:
    
    #get the labels
    mesh_labels = (ta3p100.CoarseLabelFinal & dict_key).fetch()
    
    #look for errors
    not_errors = [k for k in mesh_labels["triangles"][0] if k != 10]
    
    original_triangles = mesh[triangles][0]
    
    print(type(not_errors))
    print(len(not_errors))
    print(not_errors)
    print(type(original_triangles))
    print(len(original_triangles))
    print(original_triangles)
    
    mesh_copy['triangles'] = np.array(original_triangles[not_errors])
    

    return mesh_copy


def remove_isolated_vertices(self, mesh):
    mesh_copy = mesh.copy()

    neighborhood = self.generate_neighborhood(mesh_copy['triangles'], len(mesh_copy['vertices']))
    isolated_nodes = [portion.pop() for portion in self.get_connected_portions(neighborhood) if len(portion) == 1]

    vertices = mesh_copy['vertices']
    triangles = mesh_copy['triangles']
    vertex_list = list(vertices)

    if len(isolated_nodes) > 0:
        num_isolated_nodes_passed = 0
        isolated_nodes_set = set(isolated_nodes)
        count_to_decrement = np.zeros(len(vertices))
        for i in range(len(vertices)):
            if i in isolated_nodes_set:
                num_isolated_nodes_passed += 1
            else:
                count_to_decrement[i] = num_isolated_nodes_passed

        for i, triangle in enumerate(triangles):
            start = time.time()
            node1, node2, node3 = triangle
            triangles[i][0] -= count_to_decrement[node1]
            triangles[i][1] -= count_to_decrement[node2]
            triangles[i][2] -= count_to_decrement[node3]
        for i, isolated_node in enumerate(isolated_nodes):
            vertex_list.pop(isolated_node - i)

    mesh_copy['vertices'] = np.array(vertex_list)

    return mesh_copy


def remove_error_segments(self, key):
    full_start = time.time()

    print(key['segment_id'], key['decimation_ratio'], ":")
    start = time.time()

    mesh = (ta3p100.DecimationOrphan & key).fetch1()
    print(key['segment_id'], "mesh fetched.", time.time() - start)
    start = time.time()

    neighborhood = self.generate_neighborhood(mesh['triangles'], len(mesh['vertices']))
    print(key['segment_id'] , "neighborhood generated.", time.time() - start)
    start = time.time()

    mesh = self.remove_floating_artifacts(mesh)
    print(key['segment_id'], "floating artifacts removed.", time.time() - start)
    start = time.time()

    mesh = self.remove_isolated_vertices(mesh)
    print(key['segment_id'], "isolated nodes removed.", time.time() - start)
    start = time.time()

    key['n_vertices'] = len(mesh['vertices'])
    key['n_triangles'] = len(mesh['triangles'])
    key['vertices'] = mesh['vertices']
    key['triangles'] = mesh['triangles']

    self.insert1(key, skip_duplicates=True)
    print(key['segment_id'], "key successfully inserted.", time.time() - start)
    start = time.time()

    print("This took ", time.time() - full_start, "seconds.")
    print()








In [ ]:
def generate_neighborhood(triangles, num_vertices):
    neighborhood = dict()
    for i in range(num_vertices):
        neighborhood[i] = set()
    for node1, node2, node3 in triangles:
        neighborhood[node1].update([node2, node3])
        neighborhood[node2].update([node1, node3])
        neighborhood[node3].update([node1, node2])
    return neighborhood

def set_search_first(starting_node, neighborhood):
    """
    Modified Depth-First-Search utilizing sets to reduce duplicate checks:

    Neighborhood must be a dict with the keys being the vertex indices!
    """    
    visited_nodes = set()
    temp_stack = set()
    temp_stack.add(starting_node)
    while len(temp_stack) > 0:
        starting_node = temp_stack.pop()
        if starting_node not in visited_nodes:
            visited_nodes.add(starting_node)
            temp_stack.update(neighborhood[starting_node])
    return list(visited_nodes)

def get_connected_portions(neighborhood):
    neighborhood_copy = neighborhood.copy()
    portions = []
    while len(neighborhood_copy) > 0:
        starting_node = next(iter(neighborhood_copy))
        portion = set_search_first(starting_node, neighborhood_copy)
        for node in portion:
            neighborhood_copy.pop(node)
        portions.append(portion)
    return portions

def get_largest_portion_index(portions):
    portion_lengths = [len(portion) for portion in portions]
    return portion_lengths.index(max(portion_lengths))

def get_largest_portion(portions):
    return portions[get_largest_portion_index(portions)]

def remove_floating_artifacts(mesh):    
    mesh_copy = mesh.copy()

    """
    # Generating the neighborhoods gets quite expensive for full resolution meshes, but the searches are extremely quick.
    neighborhood = self.generate_neighborhood(mesh_copy['triangles'], len(mesh_copy['vertices']))
    portions = self.get_connected_portions(neighborhood)

    main_mesh_body_index = self.get_largest_portion_index(portions)
    triangle_removal_nodes = portions[main_mesh_body_index:] + portions[:main_mesh_body_index + 1]

    new_triangles = []
    main_body_portion = set(self.get_largest_portion(portions))
    for i, triangle in enumerate(mesh_copy['triangles']):
        node1 = triangle[0]
        if node1 in main_body_portion:
            new_triangles.append(triangle)
    mesh_copy['triangles'] = np.array(new_triangles)"""
    
    #get the triangle indices:
    
    #get the labels
    mesh_labels = (ta3p100.CoarseLabelFinal & dict_key).fetch1()
    
    #look for errors
    not_errors = [i for i,k in enumerate(mesh_labels["triangles"]) if k != 10]
    
    
    
    print(type(not_errors))
    print(len(not_errors))
    print("not_errors = "+ str(not_errors[:100]))
    original_triangles = mesh["triangles"]
    print(type(original_triangles))
    print(len(original_triangles))
    #print(original_triangles)
    print("not_errors = " + str(original_triangles[not_errors]))
    
    mesh_copy['triangles'] = np.array(original_triangles[not_errors])
    
    print("mesh_copy[Triangles]=" + str(mesh_copy['triangles'][:100]))
    print("mesh_copy[Triangles]=" + str(len(mesh_copy['triangles'])))
    
    return mesh_copy


def remove_isolated_vertices(mesh):
    mesh_copy = mesh.copy()

    neighborhood = generate_neighborhood(mesh_copy['triangles'], len(mesh_copy['vertices']))
    isolated_nodes = [portion.pop() for portion in get_connected_portions(neighborhood) if len(portion) == 1]

    vertices = mesh_copy['vertices']
    triangles = mesh_copy['triangles']
    vertex_list = list(vertices)

    if len(isolated_nodes) > 0:
        num_isolated_nodes_passed = 0
        isolated_nodes_set = set(isolated_nodes)
        count_to_decrement = np.zeros(len(vertices))
        for i in range(len(vertices)):
            if i in isolated_nodes_set:
                num_isolated_nodes_passed += 1
            else:
                count_to_decrement[i] = num_isolated_nodes_passed

        for i, triangle in enumerate(triangles):
            start = time.time()
            node1, node2, node3 = triangle
            triangles[i][0] -= count_to_decrement[node1]
            triangles[i][1] -= count_to_decrement[node2]
            triangles[i][2] -= count_to_decrement[node3]
        for i, isolated_node in enumerate(isolated_nodes):
            vertex_list.pop(isolated_node - i)

    mesh_copy['vertices'] = np.array(vertex_list)

    return mesh_copy


def remove_error_segments(key):
    full_start = time.time()

    print(key['segment_id'], key['decimation_ratio'], ":")
    start = time.time()

    mesh = (ta3p100.CleansedMesh & key).fetch1()
    print(key['segment_id'], "mesh fetched.", time.time() - start)
    start = time.time()

    original_triangles = mesh["triangles"]
    print(type(original_triangles))
    print(len(original_triangles))
    
    neighborhood = generate_neighborhood(mesh['triangles'], len(mesh['vertices']))
    print(key['segment_id'] , "neighborhood generated.", time.time() - start)
    start = time.time()

    original_triangles = mesh["triangles"]
    print(type(original_triangles))
    print(len(original_triangles))
    
    mesh = remove_floating_artifacts(mesh)
    print(key['segment_id'], "floating artifacts removed.", time.time() - start)
    start = time.time()

    mesh = remove_isolated_vertices(mesh)
    print(key['segment_id'], "isolated nodes removed.", time.time() - start)
    start = time.time()

    key['n_vertices'] = len(mesh['vertices'])
    key['n_triangles'] = len(mesh['triangles'])
    key['vertices'] = mesh['vertices']
    key['triangles'] = mesh['triangles']

    #self.insert1(key, skip_duplicates=True)
    print(key['segment_id'], "key successfully inserted.", time.time() - start)
    start = time.time()

    print("This took ", time.time() - full_start, "seconds.")
    print()
    return key








In [ ]:
segment_id = 648518346341393609
dict_key = dict(segmentation=2,segment_id=segment_id,decimation_ratio=0.35)
new_key = remove_error_segments(dict_key)


In [ ]:
write_Whole_Neuron_Off_file(new_key["segment_id"],new_key["vertices"],new_key["triangles"])



In [4]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [ ]:
#Find a suitable neurite for our neuron
synapse_dict = dict(segmentation=2,postsyn=dict_key["segment_id"])
ta3p100.Synapse() & synapse_dict

In [ ]:
ta3p100.CoarseLabelFinal()

In [ ]:
#(ta3p100.Synapse() & synapse_dict).aggr
new_seg = 648518346341393609
synapse_dict = dict(segmentation=2,postsyn=new_seg)

synapse_tables = dj.U('segmentation','presyn').aggr((ta3p100.Synapse() & synapse_dict), 
                    num_synapses='sum(postsyn = ' + str(new_seg)+ ')')
synapse_tables & "num_synapses>4"



In [ ]:
new_seg = 648518346349472574 #none
new_seg = 648518346349471562 #has like 5
synapse_dict = dict(segmentation=2,postsyn=new_seg)

synapse_tables = dj.U('segmentation','presyn').aggr((ta3p100.Synapse() & synapse_dict), 
                    num_synapses='sum(postsyn = ' + str(new_seg)+ ')')
presyn, num_synapses =(synapse_tables & "num_synapses>4").fetch("presyn","num_synapses")
print(presyn)

#find the maximum
np.argmax(num_synapses)
#max(num_synapses)


In [ ]:
#find the highest synapsing one:

#get the synapses from CoarseLabel
coarse_ids = ta3p100.CoarseLabelFinal.fetch("segment_id")

for id in coarse_ids:
    #get the table that corresponds to that id
    #(ta3p100.Synapse() & synapse_dict).aggr
    new_seg = id
    synapse_dict = dict(segmentation=2,postsyn=new_seg)

    synapse_tables = dj.U('segmentation','presyn').aggr((ta3p100.Synapse() & synapse_dict), 
                        num_synapses='sum(postsyn = ' + str(new_seg)+ ')')
    (synapse_tables & "num_synapses>10").fetch()

In [ ]:
ta3p100.Neurite()

In [ ]:
ta3p100.PairVoxelDist() & "segment_a=648518346349473597" & 'nearest_dist < 50' & ta3p100.CoarseLabelFinal.proj(segment_id="segment_b") & ta3p100.Neurite.proj(segment_id="segment_b")




In [ ]:
#possible ones:
"""
presyn           postsyn           num




"""

In [ ]:
#download the mesh of the neurite
axon_mesh = (ta3p100.Mesh & "segment_id=648518346349507130" & "segmentation=2").fetch1()
print(axon_mesh["n_vertices"])



#get the off file
write_Whole_Neuron_Off_file("axon" + str(axon_mesh["segment_id"]),axon_mesh["vertices"],axon_mesh["triangles"])

In [ ]:
#(ta3p100.Synapse() & synapse_dict).aggr
segment_id=648518346349471910
synapse_dict = dict(segmentation=2,postsyn=dict_key["segment_id"])


synapse_tables = dj.U('segmentation','presyn').aggr((ta3p100.Synapse() & synapse_dict), 
                    num_synapses='sum(postsyn = ' + str(segment_id)')')
synapse_tables & "num_synapses>4"

In [ ]:
#get the skeleton for 1) axon 648518346349507130 and 2) target 648518346341393609.
import pymeshfix
dir(pymeshfix.MeshFix)

In [ ]:
#get 648518346341393609

meshfix = pymeshfix.MeshFix(new_key["vertices"],new_key["triangles"])
meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)


In [ ]:
#meshfix.write("temp/648518346341393609_fixed.off")
#dir(meshfix.write)
write_Whole_Neuron_Off_file("target_" + str(648518346341393609),meshfix.v,meshfix.f)

In [ ]:
#get the axon

seg_id = "648518346349507130"
axon_data = (ta3p100.Mesh & "segment_id=" + str(seg_id) & "segmentation=2").fetch1()
print(len(axon_data["triangles"]))

In [ ]:

meshfix_axon = pymeshfix.MeshFix(axon_data["vertices"],axon_data["triangles"])
meshfix_axon.repair(verbose=False,joincomp=True,remove_smallest_components=False)

write_Whole_Neuron_Off_file("axon_" + str(seg_id),meshfix_axon.v,meshfix_axon.f)

In [6]:
#generate the skeletons for each of the targets

import calcification_Module as cm

In [ ]:
cm.calcification("temp/neuron_axon_648518346349507130")
cm.calcification("temp/neuron_target_648518346341393609")

In [ ]:
from meshparty import trimesh_io
mesh = trimesh_io.Mesh(vertices=axon_data["vertices"], faces=axon_data["triangles"])
voxels = mesh.voxelized(1500)
voxel_mesh = voxels.as_boxes()

In [ ]:
voxel_mesh.export("temp/neuron_axon_vox_648518346349507130.off")

In [ ]:
import pymeshfix
meshfix_axon = pymeshfix.MeshFix(voxel_mesh.vertices,voxel_mesh.faces)
meshfix_axon.repair(verbose=False,joincomp=True,remove_smallest_components=False)

write_Whole_Neuron_Off_file("axon_vox_pym_" + str(seg_id),meshfix_axon.v,meshfix_axon.f)



In [7]:
cm.calcification("temp/neur_ax_vox_meshlabserver")

0